# Notebook Overview
This notebook goes over how the training data for the U-Net was created. 
1) Raw CSV frame is converted to a 0-255 image. Fence is detected through temperature thresholding and inpainted through biharmonic inpainting.
2) Frame is put through Lucas-Kanade algorithm and if the tracking fails (i.e. if optical flow error is > 5), then frame is manually annotated.

Manual Annotation: A user clicks the approximate center of the primate’s nose. A binary mask is then generated by drawing a circle of radius 10 pixels around the selected centroid.

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import pandas as pd
from skimage import exposure
from skimage.restoration import inpaint
from skimage.restoration import inpaint_biharmonic

from PIL import Image, ImageTk
import tkinter as tk
from tkinter import filedialog



import glob
import os



In [2]:
#Pre-processing
def inpaint_biharmonic_csv(csv_img):
  data = pd.read_csv(csv_img, header=None)
  thermal_image = data.to_numpy()
  lower_threshold = 24
  upper_threshold = 28

  #Creating mask of fence: where the temperature is within the threshold will be white
  fence_mask = (thermal_image >= lower_threshold) & (thermal_image <= upper_threshold)

  #Dilating that fence-mask + Normalise:
  fence_mask = fence_mask.astype(np.uint8) * 255
  kernel = np.ones((3, 3))
  dilated_mask_temp = cv2.dilate(fence_mask, kernel, iterations=3)
  #Inpainting fence area
  inpainted_image_temp = inpaint.inpaint_biharmonic(thermal_image, dilated_mask_temp )

  inpainted_scaled = cv2.normalize(inpainted_image_temp, None, 0, 255, cv2.NORM_MINMAX)
  inpainted_scaled = inpainted_scaled.astype(np.uint8)

  return inpainted_scaled



In [3]:
def get_clicked_coordinates(image):
    if isinstance(image, np.ndarray):
        image = np.clip(image, 0, 255).astype(np.uint8)  
        image = Image.fromarray(image)

    selected_coordinates = {"x": None, "y": None}

    #creates Tkinter window
    window = tk.Tk()
    window.title("Click Once to Select a Point & Close Window")

    #create a Frame
    frame = tk.Frame(window)
    frame.pack(fill=tk.BOTH, expand=tk.YES)

    #canvas for image display
    canvas = tk.Canvas(frame, width=image.width, height=image.height)
    canvas.pack(fill=tk.BOTH, expand=tk.YES)

    #convert image to Tkinter format
    tk_image = ImageTk.PhotoImage(image)
    canvas.create_image(0, 0, anchor=tk.NW, image=tk_image)

    #Only first click is captured
    def get_coordinates(event):
        if selected_coordinates["x"] is None and selected_coordinates["y"] is None:  # Only store first click
            selected_coordinates["x"] = event.x
            selected_coordinates["y"] = event.y

            # Draw a red cross at the clicked position
            canvas.create_line(event.x - 10, event.y, event.x + 10, event.y, fill="red", width=2)  # Horizontal line
            canvas.create_line(event.x, event.y - 10, event.x, event.y + 10, fill="red", width=2)  # Vertical line
            
            print(f"Selected Coordinates: ({event.x}, {event.y})")

    canvas.bind("<Button-1>", get_coordinates)

    #User manually closes the window
    window.mainloop()

    if selected_coordinates["x"] is not None and selected_coordinates["y"] is not None:
        return (selected_coordinates["x"], selected_coordinates["y"])
    else:
        print("No point selected.")
        return None


In [4]:
#Example sequence where lucas-kanade fails/where frames can be used for training
#As this is an example it should generate very view frames for training/ in reality many more frames were given as input
#The sequence inputed cannot be the one used for testing
csv_folder_path = "example data for labelling"
csv_files = sorted(glob.glob(os.path.join(csv_folder_path, '*.csv')))
if len(csv_files) == 0:
    raise Exception("No CSV files found in the specified folder.")

#First frame inpainting
first_csv = csv_files[0]
first_frame_inpaint = inpaint_biharmonic_csv(first_csv)
frame_height, frame_width = first_frame_inpaint.shape

#Define folders where training data will be put (All will be put in desktop)
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
#failed_tracking contains inpainted and normalised frames
failed_folder = os.path.join(desktop_path, "failed_tracking")
#mask_folder contrains all the corresponding binary masks
mask_folder = os.path.join(desktop_path, "binary_masks")
#Creating the folders at the path specified above
os.makedirs(failed_folder, exist_ok=True)
os.makedirs(mask_folder, exist_ok=True)

#Creating a csv file where all training nose center coordinates are stored
coords_csv_path = os.path.join(desktop_path, "clicked_coordinates.csv")
if not os.path.exists(coords_csv_path):
    with open(coords_csv_path, "w") as f:
        f.write("frame,x,y\n")

#Manual initalisation of nose point
image = Image.fromarray(first_frame_inpaint)
x, y = get_clicked_coordinates(image)  
initial_point = np.array([[x, y]], dtype=np.float32)
p0 = initial_point.reshape(-1, 1, 2)
prev_gray = first_frame_inpaint.copy()

#Lucas–Kanade Optical Flow parameters.
lk_params = dict(
    winSize  = (15, 15),
    maxLevel = 2,
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)
)

error_threshold = 4
frame_idx = 0
tracked_coordinates = [(int(initial_point[0, 0]), int(initial_point[0, 1]))]
pixel_values = [first_frame_inpaint[int(initial_point[0, 1]), int(initial_point[0, 0])]]

for csv_file in csv_files[1:]:
    #Inpaint current frame
    inpainted_frame = inpaint_biharmonic_csv(csv_file)
    frame_gray = inpainted_frame.copy()

    #Compute optical flow from the previous frame to the current frame
    p1, st, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, p0, None, **lk_params)

    #Check for tracking failure or if error exceeds the threshold (of 5)
    if p1 is None or st[0][0] == 0 or (err is not None and err[0][0] > error_threshold):
        print(f"Tracking lost at frame index {frame_idx}.")
        print(f"Error: {err[0][0] if err is not None else 'N/A'} exceeded threshold.")

        base_name = os.path.splitext(os.path.basename(csv_file))[0]
        if frame_gray.max() <= 1.0:
            inpainted_uint8 = (frame_gray * 255).astype(np.uint8)
        else:
            inpainted_uint8 = frame_gray.astype(np.uint8)

        #Save the failed tracking image as a PNG on the Desktop/ frame used for training
        failed_png_path = os.path.join(failed_folder, base_name + ".png")
        cv2.imwrite(failed_png_path, inpainted_uint8)
        print(f"Saved failed tracking image to {failed_png_path}.")

        #Finds nose by clicking
        image_for_click = Image.fromarray(frame_gray)
        new_coords = get_clicked_coordinates(image_for_click)

        #Add nose coordiantes to csv file 
        with open(coords_csv_path, "a") as f:
            f.write(f"{base_name}.png,{new_coords[0]},{new_coords[1]}\n")
        print(f"Saved clicked coordinates for {base_name}.png to {coords_csv_path}.")

        #Create a binary mask: white circle (radius 10) on a black background (labels from the trained frame)
        mask = np.zeros(frame_gray.shape, dtype=np.uint8)
        cv2.circle(mask, (int(new_coords[0]), int(new_coords[1])), 10, 255, -1)
        mask_path = os.path.join(mask_folder, base_name + "_BIN.png")
        cv2.imwrite(mask_path, mask)
        print(f"Saved binary mask image to {mask_path}.")

        #Use the re-detected point for tracking the next frame
        re_point = np.array([[new_coords]], dtype=np.float32)
        new_point = re_point.copy()
    else:
        new_point = p1

    #Update the new tracking point.
    pt = new_point[0][0]
    tracked_coordinates.append((int(pt[0]), int(pt[1])))
    pixel_value = frame_gray[int(pt[1]), int(pt[0])]
    pixel_values.append(pixel_value)
    prev_gray = frame_gray.copy()
    p0 = new_point.copy()
    frame_idx += 1

print("Tracking complete.")


Selected Coordinates: (78, 170)
Tracking lost at frame index 9.
Error: 8.229999542236328 exceeded threshold.
Saved failed tracking image to /Users/iadician/Desktop/failed_tracking/FLIR0994_15599.png.
Selected Coordinates: (90, 178)
Saved clicked coordinates for FLIR0994_15599.png to /Users/iadician/Desktop/clicked_coordinates.csv.
Saved binary mask image to /Users/iadician/Desktop/binary_masks/FLIR0994_15599_BIN.png.
Tracking lost at frame index 10.
Error: 4.688055515289307 exceeded threshold.
Saved failed tracking image to /Users/iadician/Desktop/failed_tracking/FLIR0994_15600.png.
Selected Coordinates: (92, 180)
Saved clicked coordinates for FLIR0994_15600.png to /Users/iadician/Desktop/clicked_coordinates.csv.
Saved binary mask image to /Users/iadician/Desktop/binary_masks/FLIR0994_15600_BIN.png.
Tracking lost at frame index 28.
Error: 6.258333206176758 exceeded threshold.
Saved failed tracking image to /Users/iadician/Desktop/failed_tracking/FLIR0994_15618.png.
Selected Coordinates